In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
main_df = pd.read_csv("../data/raw/game_data_all.csv")
platform_df = pd.read_csv("../data/raw/games.csv")

In [17]:
print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]}")

Main Dataset: 67571 Second dataset: 50872


In [18]:
main_df = main_df.drop(columns=["Unnamed: 0"]) # Drop unnecessary index column
main_df = main_df.iloc[:, :4] # Keep only the first 4 columns

In [19]:
# Keeps the first occurrence of each game via link
main_df = main_df.drop_duplicates(subset='link', keep='first')

In [20]:
main_df["app_id"] = main_df["link"].str.extract(r'/app/(\d+)/')
main_df['app_id'] = main_df['app_id'].astype(int) 

In [21]:
main_df = main_df.drop(columns=['link'])

In [22]:
cols = ['app_id'] + [c for c in main_df.columns if c != 'app_id']
main_df = main_df[cols]
print(main_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 66427 entries, 0 to 67570
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   app_id        66427 non-null  int64 
 1   game          66427 non-null  object
 2   release       66427 non-null  object
 3   peak_players  66427 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.5+ MB
None


In [23]:
print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]}")

Main Dataset: 66427 Second dataset: 50872


In [24]:
print(platform_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   title           50872 non-null  object 
 2   date_release    50872 non-null  object 
 3   win             50872 non-null  bool   
 4   mac             50872 non-null  bool   
 5   linux           50872 non-null  bool   
 6   rating          50872 non-null  object 
 7   positive_ratio  50872 non-null  int64  
 8   user_reviews    50872 non-null  int64  
 9   price_final     50872 non-null  float64
 10  price_original  50872 non-null  float64
 11  discount        50872 non-null  float64
 12  steam_deck      50872 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.7+ MB
None


In [25]:
platform_df = platform_df.drop(columns=["rating", "positive_ratio", "user_reviews", "price_final", "discount", "steam_deck"])

In [26]:
print(platform_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   title           50872 non-null  object 
 2   date_release    50872 non-null  object 
 3   win             50872 non-null  bool   
 4   mac             50872 non-null  bool   
 5   linux           50872 non-null  bool   
 6   price_original  50872 non-null  float64
dtypes: bool(3), float64(1), int64(1), object(2)
memory usage: 1.7+ MB
None


In [67]:
merged_df = pd.merge(main_df, platform_df, on='app_id')
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38833 entries, 0 to 38832
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          38833 non-null  int64  
 1   game            38833 non-null  object 
 2   release         38833 non-null  object 
 3   peak_players    38833 non-null  int64  
 4   title           38833 non-null  object 
 5   date_release    38833 non-null  object 
 6   win             38833 non-null  bool   
 7   mac             38833 non-null  bool   
 8   linux           38833 non-null  bool   
 9   price_original  38833 non-null  float64
dtypes: bool(3), float64(1), int64(2), object(4)
memory usage: 2.2+ MB
None


In [66]:
print(merged_df.shape[0])

38833


In [72]:
# Find rows where release and date_release differ
differences = merged_df[merged_df["release"] != merged_df["date_release"]]

# Show first 20 differences
print(differences[["app_id", "release", "date_release"]].head(20))

# Count total differences
print("Number of differing rows:", len(differences))



     app_id     release date_release
1   2050650  2023-03-24   2023-03-23
29  1189490  2023-01-13   2023-01-12
32  1710100  2023-01-13   2023-01-12
34  1450150  2023-03-21   2021-01-02
37  1347970  2023-03-02   2021-05-07
42  2172030  2023-02-25   2023-02-24
49  2302140  2023-02-27   2023-02-26
53  2230280  2023-01-01   2022-12-31
54  2067790  2023-03-29   2022-12-07
55  1737340  2023-02-14   2023-02-13
70  1863750  2023-04-07   2022-02-06
71  1933660  2023-01-14   2023-01-13
80  2119210  2023-03-25   2022-12-08
82  1625230  2023-02-26   2021-06-04
83  2219470  2023-02-03   2023-02-02
85  1432860  2023-03-11   2023-03-10
86  1999770  2023-03-24   2023-03-23
92  1400910  2023-04-10   2021-10-11
94  1674780  2023-04-07   2022-03-27
99  1211600  2023-03-30   2023-03-29
Number of differing rows: 6720


In [56]:
game_dupes = merged_df["game"].duplicated().sum()
title_dupes = merged_df["title"].duplicated().sum()

print(f"Duplicate game names: {game_dupes}")
print(f"Duplicate title names: {title_dupes}")


Duplicate game names: 135
Duplicate title names: 108


In [73]:
merged_df = merged_df.drop(columns=["game", "date_release"])

In [74]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38833 entries, 0 to 38832
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          38833 non-null  int64  
 1   release         38833 non-null  object 
 2   peak_players    38833 non-null  int64  
 3   title           38833 non-null  object 
 4   win             38833 non-null  bool   
 5   mac             38833 non-null  bool   
 6   linux           38833 non-null  bool   
 7   price_original  38833 non-null  float64
dtypes: bool(3), float64(1), int64(2), object(2)
memory usage: 1.6+ MB
None


In [76]:
print(merged_df.head(10))

    app_id     release  peak_players                             title   win  \
0  2231450  2023-01-26          4529                       Pizza Tower  True   
1  2050650  2023-03-24        168191                   Resident Evil 4  True   
2  2324650  2023-03-31         15543  The Murder of Sonic the Hedgehog  True   
3  2263010  2023-03-28          1415                Pineapple on pizza  True   
4  1817230  2023-01-25          6132                        Hi-Fi RUSH  True   
5  2121360  2023-01-09          8672                              东北之夏  True   
6   774801  2023-04-01          8564                    Crab Champions  True   
7  2291760  2023-03-31          1227           Papa's Freezeria Deluxe  True   
8  1201270  2023-01-19           338           A Space for the Unbound  True   
9  2171690  2023-01-13           195                        Handshakes  True   

     mac  linux  price_original  
0  False  False            0.00  
1  False  False            0.00  
2   True  False  